In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda:0"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
question = "what did the baby do after he turned around near the end of the video？"
prompt = "There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child."
fact = "a) hug boy	b) flail around	c) touch the camera	d) push box to where he started	e) fall"
messages = [
    {"role": "system", "content": "你是一个描述问答推理器，请根据我给你的题目和答案，给出你选择哪一个。"},
    # {"role": "assistant", "content": "我给你的描述一定是和题目相关的，你必须给出一个选项。"},
    {"role": "user", "content": "The scenario is："+prompt+"The question is"+question+". The answer is "+fact},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print('step 3: score')
print(response)

In [ ]:
question = "what did the baby do after he turned around near the end of the video？"
prompt = "There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child."
fact = "e) fall"
messages = [
    {"role": "system", "content": "你是一个词语修改器，我给你的问题和场景描述可能存在近义词的问题，请你进行匹配，并对场景描述进行词语替换。"},
    {"role": "assistant", "content": "你需要回复一段话，是我给你的scenario的重写内容"},
    {"role": "user", "content": "The scenario is："+prompt+" The question is"+question}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print('step 3: score')
print(response)

In [ ]:
sss = """'''json
{
  "entities": [
    {
      "text": "baby",
      "type": "PERSON"
    }
  ]
}
'''"""

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "你是一个实体抽取和句法依存分析器，请基于问题，给出场景中的实体，并和题目中的实体进行匹配，给出实体内容，以问句中的实体为准，给出对应的json格式。你只能给出实体，格式如下："+sss},
    {"role": "assistant", "content": "what did the baby do after he turned around near the end of the video？"},
    {"role": "user", "content": "The scenario is：There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child."},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

In [ ]:
responses = []
responses.append(response)

In [ ]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "你只能用英语回复，你是一个场景内容推理器，请基于场景和内容，给出推理过程，给出场景关系图，给出json格式。"},
    {"role": "user", "content": "The description is：There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child. what did the baby do after he turned around near the end of the video？ "},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

{"role": "assistant", "content": f"The question is {question}. The options are {A B C D E}. 给出选择这个选项的可能的概率，从1到10给出评分，10为最有可能，1为没有可能。格式为【1/10】"},

In [ ]:
text = response
import re
import json
flag = 0
# text = text2
try:
    json_pattern = r"'''json\s*(\{(?:.|\n)*?\})\s*'''"
    matches = re.findall(json_pattern, text)
    # print(matches)
    # 假设我们只关心第一个匹配项
    json_data = json.loads(matches[0]) if matches else None
    print(json_data)
except:
    flag = 1
if flag:
  try:
      json_pattern = r'\{.*\}'
      matches = re.findall(json_pattern, text2, re.DOTALL)

      # 假设我们只关心第一个匹配项
      json_data = json.loads(matches[0]) if matches else None
      print(json_data)
  except:
      flag = 101

In [ ]:
def generate_entity(prompt, question):
    sss = """'''json
    {
    "entities": [
        {
        "text": "baby",
        "type": "PERSON"
        }
    ]
    }
    '''"""
        
    prompt = "Give me a short introduction to large language model."
    messages = [
        {"role": "system", "content": "你是一个实体抽取和句法依存分析器，请基于问题，给出场景中的实体，并和题目中的实体进行匹配，给出实体内容，以问句中的实体为准，给出对应的json格式。你只能给出实体，格式如下："+sss},
        {"role": "assistant", "content": question},
        {"role": "user", "content": "The scenario is："+prompt},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print('step 1: entity')
    print(response)

    return response

def generate_scene_graph(prompt, question):
    messages = [
    {"role": "system", "content": "你只能用英语回复，你是一个场景内容推理器，请基于场景和内容，给出推理过程，给出场景关系图，给出json格式。"},
    {"role": "user", "content": "The description is：There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child. what did the baby do after he turned around near the end of the video？ "},
]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    text = response
    import re
    import json
    flag = 0
    # text = text2
    try:
        json_pattern = r"'''json\s*(\{(?:.|\n)*?\})\s*'''"
        matches = re.findall(json_pattern, text)
        # print(matches)
        # 假设我们只关心第一个匹配项
        json_data = json.loads(matches[0]) if matches else None
        print(json_data)
    except:
        flag = 1
        if flag:
            try:
                json_pattern = r'\{.*\}'
                matches = re.findall(json_pattern, text, re.DOTALL)

                # 假设我们只关心第一个匹配项
                json_data = json.loads(matches[0]) if matches else None
                print(json_data)
            except:
                flag = 101
    print('step 2: scene graph')
    print(json_data)
    return json_data

def generate_enhancement_scene(prompt, question, fact):
    # prompt = "Give me a short introduction to large language model."
    messages = [
        {"role": "system", "content": "你是一个场景增强器，你需要根据我的问题和我给你的答案还有增强场景描述，将这句话扩展到你生成的描述中。"},
        {"role": "assistant", "content": "用英语回答，只能生成一段话"},
        {"role": "user", "content": "The scenario is："+prompt+"The question is"+question+"The answer (fact) is"+fact},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print('step 1: entity')
    print(response)

    return response

def generate_question_rank(prompt, question, fact):
    # prompt = "Give me a short introduction to large language model."
    messages = [
        {"role": "system", "content": "你是一个题目与场景比对器，根据我给你的场景描述，一步一步分析."},
        {"role": "assistant", "content": "用英语回答，从0-10进行打分，给出你认为的相关性，完全合理赋10分，完全不合理赋0分。输出格式：[5]，你的回答是什么。"},
        {"role": "user", "content": "The scenario is："+prompt+"The question is"+question+". The answer is "+fact},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print('step 3: score')
    print(response)
    
    return response

    
    
    

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda:0"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# history
question = "what did the baby do after he turned around near the end of the video？"
prompt = "There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child."
fact = "a) hug boy	b) flail around	c) touch the camera	d) push box to where he started	e) fall"
messages = [
    {"role": "system", "content": "你是一个内容理解与推理专家，你需要做的是从我的场景描述中进行内容理解和推理，请基于问题，推断出场景，并回答我的问题。"},
    {"role": "assistant", "content": "请回答我的问题：The question is"+question+". The options is "+fact+". 回答格式：【选项k)】"},
    {"role": "user", "content": "请根据我给你的内容一步一步考虑：1) 构建scene graph 2) 对每个选项进行分析，对每个选项打一个1-10的评分。3）给出最终的答案。The scenario is："+prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
# print('step 3: score')
print(response)



In [ ]:
def enhance(prompt, fact):
    question = "what did the baby do after he turned around near the end of the video？"
    prompt = "There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child."
    fact = "a) hug boy	b) flail around	c) touch the camera	d) push box to where he started	e) fall"
    messages = [
        {"role": "system", "content": "你是一个内容增强专家，你需要根据我给你的事实（问答题），对场景介绍进行增强，并最后同样生成一段话。"},
        {"role": "assistant", "content": "question: "+question+". answer: "+fact+". 请生成一段内容增强的话。"},
        {"role": "user", "content": "The scenario is："+prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # print('step 3: score')
    print(response)

In [ ]:
def response_fact(prompt, question, fact):
    question = "what did the baby do after he turned around near the end of the video？"
    prompt = "There is a little boy, a young child and his mom. The little boy is playing with a toy and wearing a tie. The young child is also playing with a toy. And, the mom is taking care of the little boy and young child."
    fact = "a) hug boy	b) flail around	c) touch the camera	d) push box to where he started	e) fall"
    messages = [
        {"role": "system", "content": "你是一个内容理解与推理专家，你需要做的是从我的场景描述中进行内容理解和推理，请基于问题，推断出场景，并回答我的问题。"},
        {"role": "assistant", "content": "请回答我的问题：The question is"+question+". The options is "+fact+". 回答格式：【选项k)】"},
        {"role": "user", "content": "请根据我给你的内容一步一步考虑：1) 构建scene graph 2) 对每个选项进行分析，对每个选项打一个1-10的评分。3）给出最终的答案。The scenario is："+prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # print('step 3: score')
    print(response)


